In [1]:
import os

In [2]:
%pwd

'c:\\Users\\asaty\\OneDrive\\Documents\\projects\\nlp_project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\asaty\\OneDrive\\Documents\\projects\\nlp_project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [6]:
from src.nlp_project.constants import *
from src.nlp_project.utils.common import read_yaml,create_directories

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config

In [9]:
import os
from src.nlp_project.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [18]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        try:
            input_encodings = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)

            with self.tokenizer.as_target_tokenizer():
                target_encodings = self.tokenizer(example_batch['summary'], max_length=128, truncation=True)

            return {
                'input_ids': input_encodings['input_ids'],
                'attention_mask': input_encodings['attention_mask'],
                'labels': target_encodings['input_ids']
            }
        except Exception as e:
            print(f"Error in convert_examples_to_features: {e}")
            return None  # You might want to handle this differently based on your use case

    def convert(self):
        try:
            dataset_samsum = load_from_disk(self.config.data_path)

            # Specify a different output directory
            output_dir = os.path.join(self.config.root_dir, "transformed_samsum_dataset")
            
            # Check if the output directory already exists
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)

            dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
            dataset_samsum_pt.save_to_disk(output_dir)
        except Exception as e:
            print(f"Error in convert: {e}")


In [20]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-02-12 12:16:47,760: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-12 12:16:47,762: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-12 12:16:47,764: INFO: common: created directory at: artifacts]
[2024-02-12 12:16:47,765: INFO: common: created directory at: artifacts/data_ingestion]


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 88787.86 examples/s] 
